In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_page_urls_array(link, maxpage):
    """in order to get the url list of each page"""
    pagination = 101
    results =[]
    results.append(link)
    while pagination <= maxpage:
        following_link = link + '/' + str(pagination)
        results.append(following_link)
        pagination += 100
    return results

In [3]:
def get_html(page_url):
    """to get the html format for each page"""
    response = requests.get(page_url)
    return response.text

In [4]:
def get_BoxOffice_ranking(urls, column_num):
    """use page urls - loop url in it - call get_html function to get the html for each page, and then extract the wated data from the html"""
    results = []
    for url in urls:
        raw_html = get_html(url) # this gives us html of a page 
        parsed_html = BeautifulSoup(raw_html, "html.parser")
        rows = parsed_html.select('center table tr')
        ths = rows[0].select('th')
        column_0 = ths[0].get_text()
        column_1 = ths[1].get_text()
        column_2_1 = ths[2].get_text()
        column_2_2 = ths[2].get_text() + "_href"
        column_3 = ths[3].get_text()
        column_4 = ths[4].get_text()
        column_5 = ths[5].get_text()
        if column_num > 6:
            column_6 = ths[6].get_text()

        for row in rows:
            #print('row is ', row)
            tds = row.select('td')
            if (len(tds) > 0):
                result = {}
                result[column_0] = tds[0].get_text()
                result[column_1] = tds[1].get_text()
                result[column_2_1] = tds[2].get_text()
                result[column_2_2] = tds[2].find("a")['href']
                result[column_3] = tds[3].get_text()
                result[column_4] = tds[4].get_text()
                result[column_5] = tds[5].get_text()
                if column_num > 6:
                    result[column_6] = tds[6].get_text()      
                results.append(result)
    return results

In [457]:
Boxoffice_first_link = "https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time"
total_pages_BO = 16901
all_Boxoffice_links = get_page_urls_array(Boxoffice_first_link,total_pages_BO)
#print(all_links)
test_Boxoffice_links = all_Boxoffice_links[0:1]
print(test_Boxoffice_links)
#这里拿到了3个links。我们做测试

['https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time']


In [458]:
column_num = 7
boxOffice_result = get_BoxOffice_ranking(test_Boxoffice_links, column_num)

In [9]:
test_Boxoffice_links[0]

'https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time'

In [10]:
Budget_first_link = "https://www.the-numbers.com/movie/budgets/all"
total_pages_BG = 6065
all_Budget_links = get_page_urls_array(Budget_first_link,total_pages_BG)
#print(all_Budget_links)
test_Budget_links = all_Budget_links[0:1]
print(test_Budget_links)
#这里拿到了3个links。我们做测试

['https://www.the-numbers.com/movie/budgets/all']


In [11]:
column_num = 6
budget_result = get_BoxOffice_ranking(test_Budget_links, column_num)

In [440]:
budget_result[0]

{'\xa0': '1',
 'ReleaseDate': 'Apr 23, 2019',
 'Movie': 'Avengers: Endgame',
 'Movie_href': '/movie/Avengers-Endgame-(2019)#tab=summary',
 'ProductionBudget': '\xa0$400,000,000',
 'DomesticGross': '\xa0$858,373,000',
 'WorldwideGross': '\xa0$2,797,800,564'}

In [474]:
def get_csv_header(dict_list):
    header= list(dict_list[0].keys())
    for item in dict_list:
        for key in item.keys():
            if key not in header:
                header.append(key)
    return header

In [475]:
def create_csv(filename, dict_list, header):
    f = open(filename,'w',encoding='utf-8')
    with f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        for i in range(len(dict_list)):
            writer.writerow(dict_list[i])
    f.close()

In [476]:
boxoffice_csv_header=get_csv_header(boxOffice_result)
create_csv("test_boxoffice.csv", boxOffice_result, boxoffice_csv_header)

In [477]:
budget_csv_header=get_csv_header(budget_result)
create_csv("test_budget.csv", budget_result, budget_csv_header)

In [479]:
def get_genres_box_links(boxOffice_result):
    genres_links = []
    for infor in boxOffice_result:
        link = "https://www.the-numbers.com" + infor["Movie_href"]
        genres_links.append(link)
    return genres_links

genres_links = get_genres_box_links(boxOffice_result)
genres_links[0:5]

['https://www.the-numbers.com/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=summary',
 'https://www.the-numbers.com/movie/Avengers-Endgame-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Avatar#tab=summary',
 'https://www.the-numbers.com/movie/Black-Panther#tab=summary',
 'https://www.the-numbers.com/movie/Avengers-Infinity-War#tab=summary']

In [580]:
results = []
raw_html = get_html("https://www.the-numbers.com/movie/Satanic#tab=summary") # this gives us html of a page 
parsed_html = BeautifulSoup(raw_html, "html.parser")
tables = parsed_html.select('#summary table')
for table in tables:
    trs = table.select('tr')
    if len(trs) > 10:
        result = {}
        for tr in trs:
            tds = tr.find_all("td")
            if len(tds) == 2:
                result["href"] = url.replace("https://www.the-numbers.com","")
                result[tds[0].get_text()] = tds[1].get_text()
        results.append(result)
print(results)

[{'href': 'h', 'Domestic Releases:': 'July 1st, 2016 (Limited) by Magnolia Pictures', 'International Releases:': 'June 26th, 2015 (Wide) (South Korea)\nSeptember 2nd, 2016 (Wide) (Argentina)\nApril 7th, 2017 (Wide), released as Å\x9eeytani (Turkey)\n', 'Video\xa0Release:': 'October 4th, 2016 by Magnolia Home Entertainment', 'MPAA\xa0Rating:': 'R for bloody horror violence, language throughout, brief nudity and some drug use.(Rating bulletin 2414 (Cert #50308), 3/2/2016)', 'Running Time:': '85 minutes', 'Comparisons:': 'vs. Rogue One: A Star Wars StoryCreate your own comparison chart…', 'Keywords:': 'Set in Los Angeles, Road Trip, Prologue, Voiceover/Narration, Retro Black and White, Intertitle, Occult, Satanism, Ouija Board, SÃ©ance, Spring Break, Non-Chronological, Surprise Twist, Runaway', 'Source:': 'Original Screenplay', 'Genre:': 'Horror', 'Production\xa0Method:': 'Live Action', 'Creative\xa0Type:': 'Fantasy', 'Production Companies:': 'Magnet Releasing, Circle of Confusion', 'Prod

In [581]:
def get_Genres_time_language(genres_links):
    """use page urls - loop url in it - call get_html function to get the html for each page, and then extract the wated data from the html"""
    results = []
    for url in genres_links:
        raw_html = get_html(url) # this gives us html of a page 
        parsed_html = BeautifulSoup(raw_html, "html.parser")
        tables = parsed_html.select('#summary table')
        for table in tables:
            trs = table.select('tr')
            if len(trs) > 10:
                result = {}
                for tr in trs:
                    tds = tr.find_all("td")
                    if len(tds) == 2:
                        result["href"] = url.replace("https://www.the-numbers.com","")
                        result[tds[0].get_text()] = tds[1].get_text()
                results.append(result)
    return  results

In [582]:
get_Genres_time_language(genres_links[1:3])

[{'href': '/movie/Avengers-Endgame-(2019)#tab=summary',
  'Domestic Releases:': 'April 26th, 2019 (Wide) by Walt DisneyApril 26th, 2019 (IMAX) by Walt Disney',
  'International Releases:': 'April 23rd, 2019 (Limited) (China)\nApril 24th, 2019 (Wide) (Australia)\nApril 24th, 2019 (Wide) (China)\nApril 24th, 2019 (Wide) (France)\nApril 24th, 2019 (Wide) (Germany)\n... Show all releases\n',
  'Video\xa0Release:': 'July 30th, 2019 by Walt Disney Home Entertainment',
  'MPAA\xa0Rating:': 'PG-13 for sequences of sci-fi violence and action, and some language.(Rating bulletin 2573 (Cert #52133), 4/10/2019)',
  'Running Time:': '181 minutes',
  'Franchise:': 'Marvel Cinematic UniverseAvengers',
  'Comparisons:': 'All-Time Top-Grossing Filmsvs. Avengers: Infinity WarCreate your own comparison chart…',
  'Keywords:': 'Ensemble, Marvel Comics, Animal Lead, Non-Chronological, Super Villains, Death of a Best Friend, Motion Capture Performance, 3-D, 3-D - Post-production Conversion, IMAX: DMR, Time T

In [368]:
budge_csv_header=get_csv_header(a)
create_csv("test111_budget.csv", a, budge_csv_header)

In [282]:
genres_links[17:18]

['https://www.the-numbers.com/movie/Star-Wars-Ep-I-The-Phantom-Menace#tab=summary']

In [583]:
def get_internal_box_links(boxOffice_result):
    international_box_links = []
    for infor in boxOffice_result:
        genres_link = "https://www.the-numbers.com" + infor["Movie_href"]
        international_box_link = genres_link.replace(genres_link[-7:], "international")
        international_box_links.append(international_box_link)
    return international_box_links
international_box_links = get_internal_box_links(boxOffice_result)
international_box_links[0:5]

['https://www.the-numbers.com/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=international',
 'https://www.the-numbers.com/movie/Avengers-Endgame-(2019)#tab=international',
 'https://www.the-numbers.com/movie/Avatar#tab=international',
 'https://www.the-numbers.com/movie/Black-Panther#tab=international',
 'https://www.the-numbers.com/movie/Avengers-Infinity-War#tab=international']

In [584]:
def get_international_box(international_box_links):
    """use page urls - loop url in it - call get_html function to get the html for each page, and then extract the wated data from the html"""
    results = []
    for url in international_box_links:
        raw_html = get_html(url) # this gives us html of a page 
        parsed_html = BeautifulSoup(raw_html, "html.parser")
        divs = parsed_html.select('#international div')
        #print(divs[0])
        rows = divs[0].find_all("tr")
        #print(trs)
        #print(rows[1].select('td'))
        result = {}
        for row in rows:
            #print('row is ', row)
            tds = row.select('td')
            if (len(tds) > 3):
                country_name = tds[0].get_text()
                result["href"] = url.replace("https://www.the-numbers.com","")
                result[country_name] = tds[6].get_text()
        results.append(result)
    return results
                    

In [386]:
b = get_international_box(international_box_links[0:3])


In [472]:
budg_csv_header=get_csv_header(b)
budg_csv_header
create_csv("test222_budget.csv", b, budg_csv_header)